In [10]:
import pandas as pd
from read_db.CH import Getch
import pandahouse
from datetime import date,timedelta, datetime
import telegram
import seaborn as sns
import matplotlib.pyplot as plt
import io
import numpy as np

sns.set_style("whitegrid")

In [14]:
from airflow.decorators import dag, task
from airflow.operators.python import get_current_context

from airflow import DAG
from airflow.operators.python_operator import PythonOperator

In [15]:
#Дефолтные параметры, которые прокидываются в таски
default_args = {
    'owner': 'e-orlova-7',
    'depends_on_past' : False,
    'retries':2,
    'retry_delay': timedelta(minutes=5),
    'start_date': datetime(2022,6,2),
}

In [16]:
#Интеравал запуска Даг
schedule_interval = '0 12 * * *'

In [17]:
dag = DAG('DAG_orlova', default_args=default_args, schedule_interval=schedule_interval)

In [20]:
full_data = Getch(
'''
select user_id,source ,gender, age, os
        ,toDate(time) as event_date
        ,countIf(post_id,action ='like') as like
        ,countIf(post_id,action='view') as view
from simulator_20220520.feed_actions
where toDate(time) = today()-1
group by user_id, source, gender, age, os, toDate(time)
'''
    ).df
full_data.head(2)


,user_id,source,gender,age,os,event_date,like,view
0,65321,ads,1,42,Android,2022-06-01,2,18
1,136108,organic,1,48,Android,2022-06-01,3,24


In [21]:
def foo1():
    feed = Getch(
        '''
        select user_id,source ,gender, age, os
                ,toDate(time) as event_date
                ,countIf(post_id,action ='like') as like
                ,countIf(post_id,action='view') as view
        from simulator_20220520.feed_actions
        where toDate(time) = today()-1
        group by user_id, source, gender, age, os, toDate(time)
        ''').df
    return feed

In [ ]:
def foo2():
def foo3():

In [22]:
t1 = PythonOperator(task_id='foo1', # Название таска
                    python_callable=foo1, # Название функции
                    dag=dag) # Параметры DAG

In [ ]:
t1 = PythonOperator(task_id='foo1', # Название таска
                    python_callable=foo1, # Название функции
                    dag=dag) # Параметры DAG

t2 = PythonOperator(task_id='foo2', # Название таска
                    python_callable=foo2, # Название функции
                    dag=dag) # Параметры DAG

t3 = PythonOperator(task_id='foo2', # Название таска
                    python_callable=foo2, # Название функции
                    dag=dag) # Параметры DAG

In [ ]:
# Python-операторы
t1 >> t2 >> t3 

In [ ]:
t1

In [ ]:
# Методы таска
t1.set_downstream(t2)
t2.set_downstream(t3)

In [ ]:
@dag(default_args=default_args, schedule_interval=schedule_interval, catchup =False)
def DAG_orlova():
    pass

In [23]:
# coding=utf-8

from datetime import datetime, timedelta
import pandas as pd
from io import StringIO
import requests

from airflow.decorators import dag, task
from airflow.operators.python import get_current_context


# Функция для CH
def ch_get_df(query='Select 1', 
              host='https://clickhouse.lab.karpov.courses', user='student', password='dpo_python_2020'):
    r = requests.post(host, data=query.encode("utf-8"), auth=(user, password), verify=False)
    result = pd.read_csv(StringIO(r.text), sep='\t')
    return result


query = """SELECT 
               toDate(time) as event_date, 
               country, 
               source,
               count() as likes
            FROM 
                simulator_20220520.feed_actions 
            where 
                toDate(time) = today()-1 
                and action = 'like'
            group by
                event_date,
                country,
                source
            format TSVWithNames"""

# Дефолтные параметры, которые прокидываются в таски
default_args = {
    'owner': 'e-orlova-7',
    'depends_on_past': False,
    'retries': 2,
    'retry_delay': timedelta(minutes=5),
    'start_date': datetime(2022, 6, 2),
}

# Интервал запуска DAG
schedule_interval = '0 23 * * *'

@dag(default_args=default_args, schedule_interval=schedule_interval, catchup=False)
def dag_sim_example():

    @task()
    def extract():
        query = """SELECT 
                       toDate(time) as event_date, 
                       country, 
                       source,
                       count() as likes
                    FROM 
                        simulator.feed_actions 
                    where 
                        toDate(time) = '2022-01-26' 
                        and action = 'like'
                    group by
                        event_date,
                        country,
                        source
                    format TSVWithNames"""
        df_cube = ch_get_df(query=query)
        return df_cube

    @task
    def transfrom_source(df_cube):
        df_cube_source = df_cube[['event_date', 'source', 'likes']]\
            .groupby(['event_date', 'source'])\
            .sum()\
            .reset_index()
        return df_cube_source

    @task
    def transfrom_countries(df_cube):
        df_cube_country = df_cube[['event_date', 'country', 'likes']]\
            .groupby(['event_date', 'country'])\
            .sum()\
            .reset_index()
        return df_cube_country

    @task
    def load(df_cube_source, df_cube_country):
        context = get_current_context()
        ds = context['ds']
        print(f'Likes per source for {ds}')
        print(df_cube_source.to_csv(index=False, sep='\t'))
        print(f'Likes per country for {ds}')
        print(df_cube_country.to_csv(index=False, sep='\t'))

    df_cube = extract()
    df_cube_source = transfrom_source(df_cube)
    df_cube_country = transfrom_countries(df_cube)
    load(df_cube_source, df_cube_country)

dag_sim_example = dag_sim_example()